In [19]:
%matplotlib inline

from copy import deepcopy
import cartopy
import datetime
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import seaborn as sns
from scipy.stats import ttest_1samp

import xarray as xr

# modules for datacube
import datacube
from datacube.utils import masking
from datacube.helpers import ga_pq_fuser

# Import external functions from dea-notebooks
sys.path.append("../../Scripts/")
sys.path.append("../Development_notebooks_BD/10_Scripts/")
import dea_plotting, dea_datahandling

# from dea_dask import create_local_dask_cluster
# import DEADataHandling #try and use old function to load, mask and concatenate C2 fractional cover

# ignore datacube warnings (needs to be last import statement)
import warnings

warnings.filterwarnings("ignore", module="datacube")

from odc.ui import with_ui_cbk

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# #initialise dask cluster
# create_local_dask_cluster()

In [21]:
# set datacube alias (just a string with what you're doing)
dc = datacube.Datacube(app="dc-FC")

## Take a look at our dataset density

define our query area + time

In [12]:
query_c2 = {
    "output_crs": "epsg:3577",
    "resolution" : (-25,25),
    "group_by": "solar_day",
     "lat": (-27.31, -27.41),
    "lon": (151.01, 151.12),
    "time": ("2018-05-01", "2018-06-01"),
}

### Check out how much data we have

In [13]:
# count the number of datasets for fc2 ls5
datasets_ls5_fc2 = dc.find_datasets(product="ls5_fc_albers", **query_c2)
# count the number of datasets for fc2 ls7
datasets_ls7_fc2 = dc.find_datasets(product="ls7_fc_albers", **query_c2)
# count the number of datasets for fc2 ls8
datasets_ls8_fc2 = dc.find_datasets(product="ls8_fc_albers", **query_c2)
print(len(datasets_ls5_fc2), len(datasets_ls7_fc2), len(datasets_ls8_fc2))

0 4 8


### Load collection 2 fractional cover

In [14]:
if datasets_ls5_fc2:
    print(f"loading {len(datasets_ls5_fc2)} ls5 fc c2")
    ds_fc2_5 = dc.load(
        product="ls5_fc_albers", 
        **query_c2      
    )
else:
    print("no ls5 fc c2")
    ds_fc2_5 = None


no ls5 fc c2


In [15]:
if datasets_ls7_fc2:
    print(f"loading {len(datasets_ls7_fc2)} ls7 fc c2")
    ds_fc2_7 = dc.load(
        product="ls7_fc_albers",
        **query_c2,
        progress_cbk=with_ui_cbk(),
    )
else:
    print("no ls7 fc c2")
    ds_fc2_7 = None

loading 4 ls7 fc c2


In [30]:
if datasets_ls8_fc2:
    print(f"loading {len(datasets_ls8_fc2)} ls8 fc c2")
    ds_fc2_8 = dc.load(
        product="ls8_fc_albers",
        **query_c2,
        progress_cbk=with_ui_cbk(),
    )
else:
    print("no ls8 fc c2")
    ds_fc2_8 = None

loading 2 ls8 fc c2


In [25]:
query_c3_ls5 = {
    "output_crs": "epsg:3577",
    "resolution" : (-25,25),
    "group_by": "solar_day",
     "lat": (-27.31, -27.41),
    "lon": (151.01, 151.12),
    "time": ("2018-05-01", "2018-06-01"),
    "platform" : "landsat-5"
    }

In [29]:
query_c3_ls7 = {
    "output_crs": "epsg:3577",
    "resolution" : (-25,25),
    "group_by": "solar_day",
     "lat": (-27.31, -27.41),
    "lon": (151.01, 151.12),
    "time": ("2018-05-01", "2018-06-01"),
    "platform" : "landsat-7"
    }

In [28]:
query_c3_ls8 = {
    "output_crs": "epsg:3577",
    "resolution" : (-25,25),
    "group_by": "solar_day",
     "lat": (-27.31, -27.41),
    "lon": (151.01, 151.12),
    "time": ("2018-05-01", "2018-06-01"),
    "platform" : "landsat-8"
    }

In [ ]:
query_c3

In [39]:
ds_fc2_8

<xarray.Dataset>
Dimensions:      (time: 4, x: 492, y: 509)
Coordinates:
  * time         (time) datetime64[ns] 2018-05-02T23:47:26.500000 ... 2018-05...
  * y            (y) float64 -3.1e+06 -3.1e+06 ... -3.112e+06 -3.112e+06
  * x            (x) float64 1.85e+06 1.85e+06 1.85e+06 ... 1.862e+06 1.862e+06
    spatial_ref  int32 3577
Data variables:
    BS           (time, y, x) int16 8 7 15 14 8 7 6 8 ... 35 40 37 29 31 9 1 16
    PV           (time, y, x) int16 45 48 44 43 48 50 49 ... 13 20 29 26 25 31
    NPV          (time, y, x) int16 46 43 40 42 43 43 44 ... 50 50 39 63 73 52
    UE           (time, y, x) int16 15 13 13 14 15 17 16 15 ... 8 8 8 9 9 6 6 8
Attributes:
    crs:           epsg:3577
    grid_mapping:  spatial_ref

### Load collection 3 fractional cover

In [31]:
# count the number of datasets for fc2 ls5
datasets_ls5_fc2 = dc.find_datasets(product="ga_ls_fc_3",  **query_c3_ls5)
# count the number of datasets for fc2 ls7
datasets_ls7_fc2 = dc.find_datasets(product="ga_ls_fc_3", **query_c3_ls7)
# count the number of datasets for fc2 ls8
datasets_ls8_fc2 = dc.find_datasets(product="ga_ls_fc_3", **query_c3_ls8)
print(len(datasets_ls5_fc2), len(datasets_ls7_fc2), len(datasets_ls8_fc2))

0 2 2


In [ ]:
# ds_fc3_ls5
# ds_fc3_ls7

In [42]:
ds_fc3_ls8 = dc.load(product="ga_ls_fc_3", 
                     like = ds_fc2_8, 
                     platform = "landsat-8",
                     progress_cbk=with_ui_cbk(),
                    )

In [ ]:
ds_fc3_ls7

ds_fc3_ls8

### Load collection 3 wofs to mask both collection 2 and 3 fractional cover (to keep comparison the same)

In [ ]:
# count the number of datasets for wofs3
datasets_wofs3 = dc.find_datasets(product="ga_ls_wo_3", **query)
len(datasets_wofs3)

In [ ]:
# load the wofs feature layers (wofls) within the same query as Fractional cover
wofls = dc.load(
    product="ga_ls_wo_3",
    **query,
    # dask_chunks={"time": 1, "x": 100, "y": 100},
    progress_cbk=with_ui_cbk()
)

### Turn the loaded wofls into a mask for fractional cover

In [ ]:
wofsmask = masking.make_mask(wofls, wet=True)
# wofsmask.water.isel(time=0).plot()

### Load collection 3 cloud mask to mask both collections 2 and 3 fractional cover

In [ ]:
c3_ard = dea_datahandling.load_ard(
    dc,
    products=["ga_ls5t_ard_3", "ga_ls7e_ard_3", "ga_ls8c_ard_3"],
    measurements=[
        "nbart_red",
        "nbart_green",
        "nbart_blue",
        "nbart_nir",
        "nbart_swir_1",
        "oa_fmask",
    ],
    **query,
    # dask_chunks={"time": 1, "x": 3000, "y": 3000}, #if you pick too small an area, this won't work
    dtype="native",
    progress_cbk=with_ui_cbk()
)

### Match timesteps for all data

In [ ]:
# # Remove milliseconds to make data comparable
ds_fc2["time"] = ds_fc2.indexes["time"].normalize()
ds_fc3["time"] = ds_fc3.indexes["time"].normalize()
wofsmask["time"] = wofsmask.indexes["time"].normalize()
c3_ard["time"] = c3_ard.indexes["time"].normalize()

In [ ]:
# Choose matching timesteps to compare
ds_compare = ds_fc3["time"].isin(ds_fc2["time"].values)
ds_fc3_sel = ds_fc3.sel(time=ds_compare)
ds_fc2_sel = ds_fc2.where(ds_fc2.time == ds_fc3.time)

In [ ]:
# free up memory
del ds_fc2_5, ds_fc2_7, ds_fc2_8
del ds_fc2, ds_fc3, ds_compare, wofls

### Rename collection 2 variables to make comparisons easier

In [ ]:
# rename collection 2 variables so we can iterate through both datasets along the same set of variables
ds_fc2_sel = ds_fc2_sel.rename_vars({"BS": "bs", "PV": "pv", "NPV": "npv", "UE": "ue"})

### work out what the nodata values are

In [ ]:
print(
    ds_fc2_sel.bs.nodata,
    ds_fc3_sel.bs.nodata,
    c3_ard.nbart_red.nodata,
    c3_ard.oa_fmask.nodata,
)

### Mask everything with everything else

In [ ]:
# replace nodata with NaNs so they plot nicely
ds_fc2_sel = ds_fc2_sel.where(ds_fc2_sel != -1)
ds_fc3_sel = ds_fc3_sel.where(ds_fc3_sel != 255)

# to compare both datasets, take data only where both sets have finite data
ds_fc2_sel = ds_fc2_sel.where(ds_fc3_sel.notnull() == True)
ds_fc3_sel = ds_fc3_sel.where(ds_fc2_sel.notnull() == True)

# turn collection 3 into a boolean mask to remove nodata in the fractional cover collections
c3_bool = c3_ard.nbart_red != c3_ard.nbart_red.nodata

# to compare both datasets, take data only where both sets have finite data
ds_fc2_sel = ds_fc2_sel.where(ds_fc3_sel.notnull())
ds_fc3_sel = ds_fc3_sel.where(ds_fc2_sel.notnull())


# mask out water from the fractional cover data
ds_fc3_sel = ds_fc3_sel.where(wofsmask.water == False)
ds_fc2_sel = ds_fc2_sel.where(wofsmask.water == False)


# #mask out clouds from the fractional cover data
if c3_ard:
    ds_fc2_masked = ds_fc2_sel.where(c3_bool == True)
    ds_fc3_masked = ds_fc3_sel.where(c3_bool == True)
else:
    print("no cloud mask  found")

### Plot Fractional Cover and Unmixing Error Bands

In [ ]:
# just a scene we want to plot - this can be done other ways
scene = 1
# get the date of the scene for our output
scene_day = np.datetime_as_string(ds_fc3_sel.isel(time=scene).time.values, unit="D")

Set up a colormap for the comparison

In [ ]:
# div_cmap = sns.diverging_palette(250, 30, l=65, center="dark", as_cmap=True)
div_pal = sns.diverging_palette(250, 30, l=65, n=21, center="dark")
# (start=2.8, rot=.1, light=0.9, n_colors=3)
div_cmap = ListedColormap(div_pal)
ticks = np.linspace(-10, 10, 1)

In [ ]:
# choose values for the comparison visualisation bounds
compmin = -10
compmax = 10

In [ ]:
# set up our images on a grid using gridspec
fig = plt.figure(figsize=(20, 18))
gs = gridspec.GridSpec(4, 3)

# plot Collection 2 Photosynthetic Veg
ax1 = plt.subplot(gs[0, 0])
ds_fc2_masked.pv.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="gist_earth_r", vmin=0, vmax=100
)
ax1.set_title("FC2 PV")

# Plot Collection 3 Photosynthetic Veg
ax1 = plt.subplot(gs[0, 1])
ds_fc3_masked.pv.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="gist_earth_r", vmin=0, vmax=100
)
ax1.set_title("FC3 PV")

# Plot the delta
ax1 = plt.subplot(gs[0, 2])
(10 > abs(ds_fc3_masked.pv.isel(time=scene) - ds_fc2_masked.pv.isel(time=scene))).plot.imshow(
    interpolation="gaussian", cmap=div_cmap, vmin=compmin, vmax=compmax
)

ax1.set_title("FC PV 3-2")

# plot Collection 2  Non-Photosynthetic Veg
ax3 = plt.subplot(gs[1, 0])
ds_fc2_masked.npv.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="copper", vmin=0, vmax=100
)
ax3.set_title("FC2 NPV")

# plot Collection 3  Non-Photosynthetic Veg
ax3 = plt.subplot(gs[1, 1])
ds_fc3_masked.npv.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="copper", vmin=0, vmax=100
)
ax3.set_title("FC3 NPV")

# Plot the delta
ax3 = plt.subplot(gs[1, 2])
(10 > abs(ds_fc3_masked.npv.isel(time=scene) - ds_fc2_masked.npv.isel(time=scene))).plot.imshow(
    interpolation="gaussian", cmap=div_cmap, vmin=compmin, vmax=compmax
)
ax3.set_title("FC NPV 3-2")

# plot Collection 2  Bare Soil
ax3 = plt.subplot(gs[2, 0])
ds_fc2_masked.bs.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="Oranges", vmin=0, vmax=100
)
ax3.set_title("FC2 BS")

# plot Collection 3  Bare Soil
ax3 = plt.subplot(gs[2, 1])
ds_fc2_masked.bs.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="Oranges", vmin=0, vmax=100
)
ax3.set_title("FC3 BS")

# Plot the delta
ax3 = plt.subplot(gs[2, 2])
(10 > abs(ds_fc3_masked.bs.isel(time=scene) - ds_fc2_masked.bs.isel(time=scene))).plot.imshow(
    interpolation="gaussian", cmap=div_cmap, vmin=compmin, vmax=compmax
)
ax3.set_title("FC BS 3-2")

# plot Collection 2  Unmixing Error
ax3 = plt.subplot(gs[3, 0])
ds_fc2_masked.ue.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="magma", vmin=0, vmax=100
)
ax3.set_title("FC2 UE")

# plot Collection 3  Unmixing Error
ax3 = plt.subplot(gs[3, 1])
ds_fc2_masked.ue.isel(time=scene).plot.imshow(
    interpolation="gaussian", cmap="magma", vmin=0, vmax=100
)
ax3.set_title("FC3 UE")

# Plot the delta
ax3 = plt.subplot(gs[3, 2])
(10 > abs(ds_fc3_masked.ue.isel(time=scene) - ds_fc2_masked.ue.isel(time=scene))).plot.imshow(
    interpolation="gaussian", cmap=div_cmap, vmin=compmin, vmax=compmax
)
ax3.set_title("FC UE 3-2")

fig.suptitle(
    f"Visually Comparing Collection 2 and 3 Fractional Cover for {scene_day}", size=18
)
gs.tight_layout(fig, rect=[0, 0, 1, 0.97])

plt.savefig(fname=f"../outputs/FC_comparison_511_{scene_day}.png")
plt.show()
None

In [ ]:
ds_fc2_masked

In [ ]:
print(scene)

In [ ]:
def scatter_fc(
    fc2_dataset, fc3_dataset, fraction: str = "pv", timestep: int = scene, **kwargs
):
    """Scatter plot collection 2 fractional cover data against collection 3
       fraction is one of pv, npv or bs """
    plt.figure(figsize=(5, 5))
    fc2 = fc2_dataset[fraction].isel(time=timestep)
    fc3 = fc3_dataset[fraction].isel(time=timestep)

    # add some noise
    fc2 = fc2 + np.random.normal(0, 0.5, size=fc2.shape)
    fc3 = fc3 + np.random.normal(0, 0.5, size=fc3.shape)
    # plt.hexbin(fc2.values.ravel(), fc3.values.ravel(), gridsize=gridsize, extent=(0,100,0,100), **kwargs)
    plt.scatter(fc2.values.ravel(), fc3.values.ravel(), s=1, **kwargs)
    plt.plot([0, 100], [0, 100], color="grey")
    plt.title(fraction)

In [ ]:
print(scene_day)

In [ ]:
for band in ["pv", "bs", "npv", "ue"]:
    scatter_fc(ds_fc2_masked, ds_fc3_masked, alpha=0.1, edgecolor="None", fraction=band)
    plt.savefig(f"../outputs/FC_comparison_band_{band}_511_{scene_day}.png")

In [ ]:
def hist_fc(
    fc2_dataset,
    fc3_dataset,
    fraction: str = "pv",
    timestep: int = scene,
    bins=100,
    **kwargs
):
    """Histogram plot collection 2 fractional cover data against collection 3
       fraction is one of pv, npv or bs """
    plt.figure(figsize=(10, 5))
    fc2 = fc2_dataset[fraction].isel(time=timestep).values
    fc3 = fc3_dataset[fraction].isel(time=timestep).values
    # getting rid of the nans
    fc2 = fc2[~np.isnan(fc2)]
    fc3 = fc3[~np.isnan(fc3)]
    # difference fc3 - fc2
    diff = fc3 - fc2
    plt.hist(diff, bins=bins, range=(-100, 100), **kwargs)
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)
    plt.axvline(mean_diff, color="k")
    plt.title(fraction)

    print(fraction, mean_diff, std_diff, ttest_1samp(diff, 0))

In [ ]:
for band in ["pv", "bs", "npv", "ue"]:
    hist_fc(ds_fc2_masked, ds_fc3_masked, fraction=band)

In [ ]:
def triangle_fc(fc2_dataset, fc3_dataset, c3_ard, timestep: int=scene, **kwargs):
    """
    """
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fc2 = fc2_dataset.isel(time=timestep)
    fc3 = fc3_dataset.isel(time=timestep)

    A2 = np.stack([fc2.pv, fc2.npv, fc2.bs])
    R = np.array([[0, 1 / 3 ** 0.5], [-0.5, -1 / 3 ** 0.5,], [0.5, -1 / 3 ** 0.5]])
    A3 = np.stack([fc3.pv, fc3.npv, fc3.bs])

    P2 = np.tensordot(R, A2, (0,0)) 
    P3 = np.tensordot(R, A3, (0,0))  

    key_points = {
        'pv': np.array([1, 0, 0]),
        'npv': np.array([0, 1, 0]),
        'bs': np.array([0, 0, 1]),
    }
    for name, kp in key_points.items():
        transformed = kp @ R * 100
        offset = (kp * 20) @ R
        for ax in axs:
            ax.scatter(transformed[0], transformed[1], s=10, c='k', zorder=10)
            ax.annotate(name, transformed, xytext=offset, textcoords='offset points', ha='center', va='center')
    
    c3_ard = c3_ard.sel(time=fc3.time)
    colours = np.transpose(
        np.stack([c3_ard.nbart_swir_1.values, c3_ard.nbart_nir.values, c3_ard.nbart_green.values]),
        (1, 2, 0)).reshape((-1, 3)).astype(float)
    
    pc_min, pc_max = np.percentile(colours, [2, 98], axis=0, keepdims=True)
    colours = np.clip(colours, pc_min, pc_max)
    colours = colours - colours.min()
    colours /= colours.max()
    
    # add some noise
    P2 = P2 + np.random.normal(0, 0.5, size=P2.shape)
    P3 = P3 + np.random.normal(0, 0.5, size=P3.shape)
    axs[0].scatter(P2[0].ravel(),P2[1].ravel(), s=1, c=colours, **kwargs)
    axs[1].scatter(P3[0].ravel(),P3[1].ravel(), s=1, c=colours, **kwargs)
    axs[2].scatter((P3 - P2)[0].ravel(),(P3 - P2)[1].ravel(), s=1, c=colours, **kwargs)

    for ax in axs:
        ax.set_xlim(-70, 70)
        ax.set_ylim(-70, 70)
        ax.set_aspect('equal')
    
    for ax, title in zip(axs, ['C2', 'C3', 'C3 - C2']):
        ax.set_title(title)

In [ ]:
triangle_fc(ds_fc2_masked,ds_fc3_masked, c3_ard, alpha=0.1, edgecolor='None')

In [ ]:
dea_plotting.rgb(c3_ard.sel(time=ds_fc3_masked.isel(time=scene).time), bands=['nbart_swir_1', 'nbart_nir', 'nbart_green'])